In [ ]:
import os
import subprocess
import pandas as pd
import json
import time

# ============= CONFIG ============= #
XLSX_PATH = "Top500Website.xlsx"     
ROOT_DOMAIN_COLUMN = "Root Domain"  
CRAWLER_PATH = "Source/Crawler"
DETECTOR_PATH = "Source/Detector"
RESULTS_PATH = "Source/Results"
MAX_WEBSITES = 400  
# ================================== #


def normalize_domain(domain: str) -> str:
    domain = domain.strip()
    if domain.startswith("http://") or domain.startswith("https://"):
        return domain
    return "https://" + domain


def domain_to_folder(domain: str) -> str:
    """Convert domain like 'www.bbc.com' into folder name: www_bbc_com."""
    return (
        domain.replace("https://", "")
            .replace("http://", "")
            .replace(".", "_")
    )


# Retry Mechanism for Crawler & Detector 
def run_crawler_with_retry(url, retries=1):
    for attempt in range(1, retries + 1):
        print(f"\n🔁 Crawler attempt {attempt}/{retries} for {url}")
        result = subprocess.run(["node", "app.js", url], cwd=CRAWLER_PATH)
        if result.returncode == 0:
            print("✅ Crawler success")
            return True
        print("❌ Crawler failed, retrying...")
        time.sleep(2)
    print("⛔ Crawler failed after 3 attempts")
    return False


def run_detector_with_retry(folder_path, retries=3):
    for attempt in range(1, retries + 1):
        print(f"\n🔁 Detector attempt {attempt}/{retries} for {folder_path}")
        result = subprocess.run(["node", "app.js", folder_path], cwd=DETECTOR_PATH)
        if result.returncode == 0:
            print("✅ Detector success")
            return True
        print("❌ Detector failed, retrying...")
        time.sleep(2)
    print("⛔ Detector failed after 3 attempts")
    return False


# Inline Analyzer
def parse_results_inlined(results_dir="Source/Results"):
    data = []
    website_dirs = [
        d for d in os.listdir(results_dir)
        if os.path.isdir(os.path.join(results_dir, d))
    ]
    for site_dir in website_dirs:
        result_path = os.path.join(results_dir, site_dir, "result.json")
        if not os.path.exists(result_path):
            print(f"⚠️ result.json NOT found for {site_dir}, skipping.")
            continue
        try:
            with open(result_path, "r") as f:
                result = json.load(f)
        except json.JSONDecodeError:
            print(f"⚠️ JSON decode error for {site_dir}, skipping.")
            continue
        # Extract violation counts
        id_leaking_count = len(result.get("idLeaking", []))
        cookie_sync_count = len(result.get("cookieSync", []))
        fingerprinting_obj = result.get("fingerprinting", {})
        fingerprinting_count = len(fingerprinting_obj.get("functions", []))
        total_violations = (
                id_leaking_count +
                cookie_sync_count +
                fingerprinting_count
        )
        is_compliant = (total_violations == 0)
        website_name = site_dir.replace("www_", "").replace("_", ".")
        data.append({
            "Website": website_name,
            "ID_Leaking_Count": id_leaking_count,
            "Cookie_Sync_Count": cookie_sync_count,
            "Fingerprinting_Count": fingerprinting_count,
            "Total_Violations": total_violations,
            "Is_Compliant": is_compliant
        })
    return pd.DataFrame(data)


def main():
    print("📄 Reading Excel file:", XLSX_PATH)
    df = pd.read_excel(XLSX_PATH)
    if ROOT_DOMAIN_COLUMN not in df.columns:
        print(f"❌ ERROR: Column '{ROOT_DOMAIN_COLUMN}' not found in Excel!")
        return
    domains = df[ROOT_DOMAIN_COLUMN].dropna().astype(str).tolist()
    domains = domains[:MAX_WEBSITES]
    print(f"🌐 Loaded {len(domains)} domains (processing first {MAX_WEBSITES}).")
    os.makedirs(RESULTS_PATH, exist_ok=True)
    # Process each website
    for i, domain in enumerate(domains, 1):
        print("\n============================")
        print(f"🚀 Processing website {i}/{MAX_WEBSITES}: {domain}")
        print("============================")
        url = normalize_domain(domain)
        folder_name = domain_to_folder(domain)
        website_result_dir = os.path.join(RESULTS_PATH, folder_name)
        os.makedirs(website_result_dir, exist_ok=True)
        result_json_path = os.path.join(website_result_dir, "result.json")
        if os.path.exists(result_json_path):
            print(f"⏭️ Skipping {domain} (result.json already exists)")
            continue
        # --- Step 1: Crawler with Retry ---
        if not run_crawler_with_retry(url):
            print(f"🚫 Skipping {domain} due to Crawler failure")
            continue
        time.sleep(2)
        # --- Step 2: Detector with Retry ---
        detector_path = "../Results/" + folder_name
        if not run_detector_with_retry(detector_path):
            print(f"🚫 Skipping {domain} due to Detector failure")
            continue
        time.sleep(1)
    # Final Analysis
    print("\n📊 Running final analysis…\n")
    df_result = parse_results_inlined(RESULTS_PATH)
    if not df_result.empty:
        output_path = "Result.xlsx"
        df_result.to_excel(output_path, index=False)
        print(f"✅ Result saved to: {output_path}")
        print("\n--- Compliance Summary ---")
        print(df_result)
        compliant = df_result["Is_Compliant"].sum()
        total = len(df_result)
        non_compliant = total - compliant
        print(f"\nTotal Sites: {total}")
        print(f"Compliant: {compliant}")
        print(f"Non-Compliant: {non_compliant}")
        print(f"Non-Compliance Rate: {non_compliant / total * 100:.2f}%")
    else:
        print("⚠️ No valid result.json files found.")
    print("\n🎉 ALL DONE!")


if __name__ == "__main__":
    main()

📄 Reading Excel file: Top500Website.xlsx
🌐 Loaded 400 domains (processing first 400).

🚀 Processing website 1/400: www.google.com
⏭️ Skipping www.google.com (result.json already exists)

🚀 Processing website 2/400: www.blogger.com
⏭️ Skipping www.blogger.com (result.json already exists)

🚀 Processing website 3/400: youtube.com
⏭️ Skipping youtube.com (result.json already exists)

🚀 Processing website 4/400: linkedin.com
⏭️ Skipping linkedin.com (result.json already exists)

🚀 Processing website 5/400: support.google.com
⏭️ Skipping support.google.com (result.json already exists)

🚀 Processing website 6/400: cloudflare.com
⏭️ Skipping cloudflare.com (result.json already exists)

🚀 Processing website 7/400: microsoft.com
⏭️ Skipping microsoft.com (result.json already exists)

🚀 Processing website 8/400: apple.com
⏭️ Skipping apple.com (result.json already exists)

🚀 Processing website 9/400: en.wikipedia.org
⏭️ Skipping en.wikipedia.org (result.json already exists)

🚀 Processing website 

Error at https://bp.blogspot.com:  Error: net::ERR_NAME_NOT_RESOLVED at https://bp.blogspot.com
    at navigate (/Users/qie/Desktop/consent-guard-main/Source/Crawler/node_modules/puppeteer/lib/cjs/puppeteer/common/FrameManager.js:155:23)
    at process.processTicksAndRejections (node:internal/process/task_queues:105:5)
    at async FrameManager.navigateFrame (/Users/qie/Desktop/consent-guard-main/Source/Crawler/node_modules/puppeteer/lib/cjs/puppeteer/common/FrameManager.js:130:21)
    at async Frame.goto (/Users/qie/Desktop/consent-guard-main/Source/Crawler/node_modules/puppeteer/lib/cjs/puppeteer/common/FrameManager.js:500:16)
    at async Page.goto (/Users/qie/Desktop/consent-guard-main/Source/Crawler/node_modules/puppeteer/lib/cjs/puppeteer/common/Page.js:1167:16)
    at async Scrape (/Users/qie/Desktop/consent-guard-main/Source/Crawler/driver/scrape.js:40:5)
    at async CollectTraces (/Users/qie/Desktop/consent-guard-main/Source/Crawler/app.js:32:24)
    at async main (/Users/qie

⛔ Crawler failed after 3 attempts
🚫 Skipping bp.blogspot.com due to Crawler failure

🚀 Processing website 18/400: sites.google.com
⏭️ Skipping sites.google.com (result.json already exists)

🚀 Processing website 19/400: googleusercontent.com
⏭️ Skipping googleusercontent.com (result.json already exists)

🚀 Processing website 20/400: accounts.google.com
⏭️ Skipping accounts.google.com (result.json already exists)

🚀 Processing website 21/400: t.me
⏭️ Skipping t.me (result.json already exists)

🚀 Processing website 22/400: europa.eu
⏭️ Skipping europa.eu (result.json already exists)

🚀 Processing website 23/400: plus.google.com
⏭️ Skipping plus.google.com (result.json already exists)

🚀 Processing website 24/400: whatsapp.com
⏭️ Skipping whatsapp.com (result.json already exists)

🚀 Processing website 25/400: adobe.com
⏭️ Skipping adobe.com (result.json already exists)

🚀 Processing website 26/400: facebook.com
⏭️ Skipping facebook.com (result.json already exists)

🚀 Processing website 27/

Error at https://uol.com.br:  TimeoutError: Navigation timeout of 40000 ms exceeded
    at /Users/qie/Desktop/consent-guard-main/Source/Crawler/node_modules/puppeteer/lib/cjs/puppeteer/common/LifecycleWatcher.js:106:111


[ERROR] No traces collected
❌ Crawler failed, retrying...
⛔ Crawler failed after 3 attempts
🚫 Skipping uol.com.br due to Crawler failure

🚀 Processing website 29/400: istockphoto.com
⏭️ Skipping istockphoto.com (result.json already exists)

🚀 Processing website 30/400: vimeo.com
⏭️ Skipping vimeo.com (result.json already exists)

🚀 Processing website 31/400: vk.com
⏭️ Skipping vk.com (result.json already exists)

🚀 Processing website 32/400: github.com
⏭️ Skipping github.com (result.json already exists)

🚀 Processing website 33/400: amazon.com
⏭️ Skipping amazon.com (result.json already exists)

🚀 Processing website 34/400: search.google.com
⏭️ Skipping search.google.com (result.json already exists)

🚀 Processing website 35/400: bbc.co.uk
⏭️ Skipping bbc.co.uk (result.json already exists)

🚀 Processing website 36/400: google.de
⏭️ Skipping google.de (result.json already exists)

🚀 Processing website 37/400: live.com
⏭️ Skipping live.com (result.json already exists)

🚀 Processing websit

...

Operation failed for ../Results/e-monsite_com: Error: ENOENT: no such file or directory, open '../Results/e-monsite_com/cookies.json'
    at Object.readFileSync (node:fs:441:20)
    at LoadData (/Users/qie/Desktop/consent-guard-main/Source/Detector/helpers/storage.js:13:21)
    at Analyse (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:21:30)
    at ProcessWebsite (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:83:24)
    at main (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:121:28)
    at Object.<anonymous> (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:142:3)
    at Module._compile (node:internal/modules/cjs/loader:1706:14)
    at Object..js (node:internal/modules/cjs/loader:1839:10)
    at Module.load (node:internal/modules/cjs/loader:1441:32)
    at Function._load (node:internal/modules/cjs/loader:1263:12) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'open',
  path: '../Results/e-monsite_com/cookies.json'
}
Fatal error: Asse


🔁 Detector attempt 3/3 for ../Results/e-monsite_com
Application Started: [ 11/30/2025, 10:59:01 PM ]

[INFO] Processing ../Results/e-monsite_com
❌ Detector failed, retrying...


Operation failed for ../Results/e-monsite_com: Error: ENOENT: no such file or directory, open '../Results/e-monsite_com/cookies.json'
    at Object.readFileSync (node:fs:441:20)
    at LoadData (/Users/qie/Desktop/consent-guard-main/Source/Detector/helpers/storage.js:13:21)
    at Analyse (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:21:30)
    at ProcessWebsite (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:83:24)
    at main (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:121:28)
    at Object.<anonymous> (/Users/qie/Desktop/consent-guard-main/Source/Detector/app.js:142:3)
    at Module._compile (node:internal/modules/cjs/loader:1706:14)
    at Object..js (node:internal/modules/cjs/loader:1839:10)
    at Module.load (node:internal/modules/cjs/loader:1441:32)
    at Function._load (node:internal/modules/cjs/loader:1263:12) {
  errno: -2,
  code: 'ENOENT',
  syscall: 'open',
  path: '../Results/e-monsite_com/cookies.json'
}
Fatal error: Asse

⛔ Detector failed after 3 attempts
🚫 Skipping e-monsite.com due to Detector failure

🚀 Processing website 299/400: hubspot.com

🔁 Crawler attempt 1/1 for https://hubspot.com
Application Started: [ 11/30/2025, 10:59:03 PM ]

[INFO] Crawling https://hubspot.com
[INFO] Created browser instance
[INFO] Visiting website...


KeyboardInterrupt: 